<a href="https://colab.research.google.com/github/the-SQuAD-squad/QA/blob/huggingface/QA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
#@title Init { form-width: "25%" }
import os
import random
import math
import numpy as np
import tensorflow as tf
import json
import pandas as pd
import re
import string
import tensorflow_hub as hub
from tensorflow import keras
from tensorflow.keras import layers

import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

!pip install transformers
import transformers
from transformers import AutoTokenizer
pd.set_option('display.max_colwidth', -1)

# fix random seeds
seed_value = 42 #@param {type:"integer"}

os.environ['PYTHONHASHSEED']=str(seed_value)
random.seed(seed_value)
np.random.seed(seed_value)

tf.compat.v1.set_random_seed(seed_value)

session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
tf.compat.v1.keras.backend.set_session(sess)

# BERT params
max_seq_length = 512
# Huggingface bert and associated tokenizer
pretrained_model_str = "roberta-base"
bert_hf_layer = transformers.TFRobertaModel.from_pretrained(
    pretrained_model_str, output_attentions=True)

#pretrained_model_str = "allenai/longformer-base-4096"
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_str)

#bert_hf_layer = transformers.TFLongformerModel.from_pretrained(
 #   pretrained_model_str, output_attentions=True)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


# Dataset creation

In [5]:
#@title df creation { form-width: "25%" }

# the official dataset is identical to the provided one
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json -O training_set.json

with open("training_set.json", "r") as f:
    json_file = json.load(f)
data = json_file["data"]

rows = []
for document in data:
  for par in document['paragraphs']:
    for qas in par['qas']:

      if len(qas['answers']) == 0: #no answer
          ans_start = -1
          ans_end = -1
          ans_text = ""
      else:
          ans_start = qas['answers'][0]['answer_start']
          ans_end = ans_start + len(qas['answers'][0]['text'])
          ans_text = qas['answers'][0]['text']
          
      rows.append({
        'id' : qas['id'],
        'title': document["title"],
        'passage': par['context'],
        'question' : qas['question'],
        'answer_idx' : (ans_start, ans_end),
        'answer_text' : ans_text
      })

df_original = pd.DataFrame(rows)

--2021-03-02 16:59:26--  https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.110.153, 185.199.108.153, 185.199.109.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.110.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42123633 (40M) [application/json]
Saving to: ‘training_set.json’

training_set.json   100%[===================>]  40.17M  62.9MB/s    in 0.6s    

2021-03-02 16:59:27 (62.9 MB/s) - ‘training_set.json’ saved [42123633/42123633]



In [11]:
df_original.head()

,id,title,passage,question,answer_idx,answer_text
0,56be85543aeaaa14008c9063,Beyoncé,"Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles ""Crazy in Love"" and ""Baby Boy"".",When did Beyonce start becoming popular?,"(269, 286)",in the late 1990s
1,56be85543aeaaa14008c9065,Beyoncé,"Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles ""Crazy in Love"" and ""Baby Boy"".",What areas did Beyonce compete in when she was growing up?,"(207, 226)",singing and dancing
2,56be85543aeaaa14008c9066,Beyoncé,"Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles ""Crazy in Love"" and ""Baby Boy"".",When did Beyonce leave Destiny's Child and become a solo singer?,"(526, 530)",2003
3,56bf6b0f3aeaaa14008c9601,Beyoncé,"Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles ""Crazy in Love"" and ""Baby Boy"".",In what city and state did Beyonce grow up?,"(166, 180)","Houston, Texas"
4,56bf6b0f3aeaaa14008c9602,Beyoncé,"Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 numbe

In [ ]:
#@title clean dataset { form-width: "25%" }

!gcloud config set project feisty-mechanic-221914
!gsutil cp gs://squad_squad/error_IDs.txt ./error_IDs.txt

with open("error_IDs.txt", "r") as f:
    unwanted_id = f.read()

unwanted_id = unwanted_id.split("\n")[:-1]
df_bert = df_original.set_index('id')
df_bert = df_bert.drop(unwanted_id)
df_bert.head()

In [16]:
def preprocess_bert(text):
    #preprocesed_text = [" ".join(str(line).split()) for line in text]
    tokenized_text = tokenizer(text, return_offsets_mapping=True)

    rows_out  = [{'input_ids': tokenized_text.input_ids[i],
                  'offsets': tokenized_text.offset_mapping[i]} for i in range(len(text))]

    return rows_out

preprocess_bert(["hi mum", "hi dad"])


[{'input_ids': [0, 3592, 8562, 2],
  'offsets': [(0, 0), (0, 2), (3, 6), (0, 0)]},
 {'input_ids': [0, 3592, 4252, 2],
  'offsets': [(0, 0), (0, 2), (3, 6), (0, 0)]}]

In [5]:
tokenizer("hi mum", "hi dad",return_offsets_mapping=True)

{'input_ids': [0, 3592, 8562, 2, 2, 3592, 4252, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1], 'offset_mapping': [(0, 0), (0, 2), (3, 6), (0, 0), (0, 0), (0, 2), (3, 6), (0, 0)]}

In [33]:
list(range(-1,-1))

[]

In [44]:
#@title BERT preprocessing { form-width: "1%" }

def preprocess_bert(text):
    tokenized_text = tokenizer(list(text), return_offsets_mapping=True)

    rows_out  = [{'input_ids': tokenized_text.input_ids[i],
                  'offsets': tokenized_text.offset_mapping[i]} for i in range(len(text))]

    return rows_out

def labeling(df):
    skip = []
    ans_token_start = []
    ans_token_end = []
    input_word_ids = []
    input_type_ids = []
    input_mask = []
    context_token_to_char = []

    for id in tqdm(df.index):

        answer = " ".join(str(df.loc[id]['answer_text']).split())
        tokenized_context = df.loc[id]['passage']
        tokenized_question = df.loc[id]['question']

        # mark all the character indexes in context that are also in answer     
        is_char_in_ans = [0] * len(df_bert.loc[id]['passage'])
        for idx in range(*df.loc[id]['answer_idx']):
            is_char_in_ans[idx] = 1


        ans_token_idx = []
        if df.loc[id]['answer_idx'] == (-1,-1):
            ans_token_idx.append(0)
        else:
        # find all the tokens that are in the answers
            for idx, (start, end) in enumerate(tokenized_context["offsets"]): #start is index of the first character of the word, end is the index of the last character of the word
                if sum(is_char_in_ans[start:end]) > 0:
                    ans_token_idx.append(idx)
            if len(ans_token_idx) == 0:
                skip.append(id)
                continue
        # create inputs as usual
        input_ids = tokenized_context['input_ids'] + tokenized_question['input_ids'][1:] #removing CLS from the beginning of the question 
        token_type_ids = [0] * len(tokenized_context['input_ids']) + [1] * len(tokenized_question['input_ids'][1:])
        attention_mask = [1] * len(input_ids)
        padding_length = max_seq_length - len(input_ids)
        
        # add padding if necessary
        if padding_length > 0:
            input_ids = input_ids + ([0] * padding_length)
            attention_mask = attention_mask + ([0] * padding_length)
            token_type_ids = token_type_ids + ([0] * padding_length)
        elif padding_length < 0:
            skip.append(id)
            continue
        input_word_ids.append(np.array(input_ids))
        input_type_ids.append(np.array(token_type_ids))
        input_mask.append(np.array(attention_mask))
        context_token_to_char.append(np.array(tokenized_context["offsets"]))
        ans_token_start.append(ans_token_idx[0])
        ans_token_end.append(ans_token_idx[-1])

    df = df.drop(skip)
    df['input_word_ids'] = input_word_ids
    df['input_type_ids'] = input_type_ids
    df['input_mask'] = input_mask
    df['context_token_to_char'] = context_token_to_char
    df['ans_token_start'] = ans_token_start
    df['ans_token_end'] = ans_token_end

    return df    

df_bert_preprocessed = df_bert.copy()
# pre-process passage and question text
print("Preprocessing passage...")
df_bert_preprocessed['passage'] = preprocess_bert(df_bert['passage'])
print("Preprocessing question...")
df_bert_preprocessed['question'] = preprocess_bert(df_bert['question'])
print("Building attention masks...")
df_bert_preprocessed = labeling(df_bert_preprocessed)


Preprocessing passage...
Preprocessing question...
Building attention masks...


In [47]:
len(df_bert_preprocessed[df_bert_preprocessed["ans_token_start"] == 0])

43423

In [41]:
tokenizer.decode(df_bert_preprocessed.iloc[0]["passage"]['input_ids'])

'<s>Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".</s>'

In [36]:
df_bert_preprocessed.iloc[0]

title                    Beyoncé                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [ ]:
preprocessed_lenghts = []
for i in range(len(df_bert_preprocessed)):
    preprocessed_lenghts.append(np.count_nonzero(df_bert_preprocessed.iloc[i]["input_word_ids"]))



#Model

In [48]:
#@title split { form-width: "25%" }

split_value = 0.1 #@param {type:"number"} 
val_dim = int(len(df_bert_preprocessed['title'].unique()) * split_value)
val_titles = np.random.choice(df_bert_preprocessed['title'].unique(), size=val_dim, replace=False)

df_bert_val = df_bert_preprocessed[df_bert_preprocessed['title'].isin(val_titles)]
df_bert_train = df_bert_preprocessed[~(df_bert_preprocessed['title'].isin(val_titles))]

x_train, y_train = [np.stack(df_bert_train["input_word_ids"]),np.stack(df_bert_train["input_mask"]),np.stack(df_bert_train["input_type_ids"])],[np.stack(df_bert_train["ans_token_start"]),np.stack(df_bert_train["ans_token_end"])]
x_eval, y_eval = [np.stack(df_bert_val["input_word_ids"]),np.stack(df_bert_val["input_mask"]),np.stack(df_bert_val["input_type_ids"])],[np.stack(df_bert_val["ans_token_start"]),np.stack(df_bert_val["ans_token_end"])]

In [50]:
#@title model definition { form-width: "25%" }

input_word_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name='input_word_ids')
input_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name='input_mask')
input_type_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name='input_type_ids')

#pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, input_type_ids])

#HUGGINGFACE 🤗🤗🤗🤗🤗🤗🤗🤗🤗🤗🤗🤗🤗
sequence_output = bert_hf_layer(input_ids=input_word_ids, attention_mask=input_mask, 
                                token_type_ids=input_type_ids).last_hidden_state

#do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()

start_logits = layers.Dense(1, name="start_logit", use_bias=False)(sequence_output)
start_logits = layers.Flatten(name="flatten_start")(start_logits)

end_logits = layers.Dense(1, name="end_logit", use_bias=False)(sequence_output)
end_logits = layers.Flatten(name="flatten_end")(end_logits)

start_probs = layers.Activation(keras.activations.softmax, name="softmax_start")(start_logits)
end_probs = layers.Activation(keras.activations.softmax, name="softmax_end")(end_logits)

model = keras.Model(inputs=[input_word_ids, input_mask, input_type_ids], 
                    outputs=[start_probs, end_probs],
                    name="BERT_QA")

loss = keras.losses.SparseCategoricalCrossentropy(from_logits=False)

optimizer = keras.optimizers.Adam(lr=1e-5, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

model.summary(line_length=150)

Model: "BERT_QA"
______________________________________________________________________________________________________________________________________________________
Layer (type)                                     Output Shape                     Param #           Connected to                                      
input_word_ids (InputLayer)                      [(None, 512)]                    0                                                                   
______________________________________________________________________________________________________________________________________________________
input_mask (InputLayer)                          [(None, 512)]                    0                                                                   
______________________________________________________________________________________________________________________________________________________
input_type_ids (InputLayer)                      [(None, 512)]               

In [51]:
#@title metrics { form-width: "25%" }

def prec(y_true, y_pred):
    sampled = tf.argmax(y_pred, axis=-1)
    return 1 - tf.math.count_nonzero(tf.squeeze(tf.cast(y_true, tf.int64)) - sampled) / tf.cast(len(sampled), tf.int64)

def dist(y_true, y_pred):
    sampled = tf.argmax(y_pred, axis=-1)
    return tf.reduce_sum(tf.abs(tf.squeeze(tf.cast(y_true, tf.int64)) - sampled)) / tf.cast(len(sampled), tf.int64)

In [ ]:
#@title train { form-width: "25%" }
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
batch_size = 8
epochs = 200
steps_per_epoch = 20
saveDir = os.path.join(os.getcwd(), 'saved_models')
if not os.path.isdir(saveDir):
    os.makedirs(saveDir)
chkpt = saveDir + '/squad_check.hdf5'

ENABLE_WANDB = True        #@param {type:"boolean"}
wandb_experiment_name = "HF_RoBERTa_SQuAD_V2.0" #@param {type: "string"}
#if ENABLE_WANDB:
    #!pip install wandb > /dev/null
    #!wandb login
    #import wandb
    #from wandb.keras import WandbCallback
    #wandb.init(project="SQUAD", name=wandb_experiment_name)
    #wandb.config.batch_size = batch_size
    #wandb.config.epochs = epochs
    
es_cb = EarlyStopping(monitor='val_loss', patience=2,verbose=1, mode='auto')
cp_cb = ModelCheckpoint(filepath = chkpt, monitor='val_loss', verbose=1, 
                        save_best_only=False, mode='auto', 
                        save_weights_only=True)

callbacks = [es_cb, cp_cb]

if ENABLE_WANDB:
    callbacks.append(WandbCallback(log_batch_frequency=10,
                                   save_weights_only=True))

tf.keras.backend.clear_session()

model.compile(optimizer=optimizer, loss=[loss,loss], metrics=[prec,dist])
history = model.fit(x_train, y_train, epochs=epochs,callbacks=callbacks, 
                    validation_data=(x_eval,y_eval),batch_size=batch_size)#,
                    #steps_per_epoch = steps_per_epoch)

Epoch 1/200
  332/14645 [..............................] - ETA: 2:08:28 - loss: 0.8577 - softmax_start_loss: 0.4465 - softmax_end_loss: 0.4112 - softmax_start_prec: 0.8334 - softmax_start_dist: 4.1069 - softmax_end_prec: 0.8613 - softmax_end_dist: 4.3449

#Evaluation

In [56]:
#@title download best weights
!wget https://api.wandb.ai/files/buio/SQUAD/184b7gum/model-best.h5

model.load_weights("model-best.h5")

--2021-03-02 18:04:08--  https://api.wandb.ai/files/buio/SQUAD/184b7gum/model-best.h5
Resolving api.wandb.ai (api.wandb.ai)... 35.186.228.49
Connecting to api.wandb.ai (api.wandb.ai)|35.186.228.49|:443... connected.
HTTP request sent, awaiting response... 307 Temporary Redirect
Location: https://storage.googleapis.com/wandb-production.appspot.com/buio/SQUAD/184b7gum/model-best.h5?Expires=1614708308&GoogleAccessId=wandb-production%40appspot.gserviceaccount.com&Signature=Dgj3wL37Cfy5rNWCr5HcVZURENoAt569Ca30b706E%2B4%2BrwlX%2FStJ5x0hhWbqQuuiwNvca0p4LeDZxqdTPHv1qejLQTIGb3DJQdhAL5tQeNqF6WklRZ0S2Pmuou%2FZhZlxq7eF2K40heoKWH5651t2qB%2Bo9kOqN29O3FzlAjjb6eFgHkKs83zBHpym%2Bw2f8JRkRA0XDN6iTeG%2FDoQnG6L5IX07pA7mGQSAMlBUNpjprgt63ZkOxdY8qcZo04H1Er6TooGszgwse0BI6icUNxFuNK%2B4SeLyuR8GmGnPGmvQpDPEByfM20J8DFyt7lfqxTIuFqVoYtjtOhNND68DuGI56Q%3D%3D [following]
--2021-03-02 18:04:08--  https://storage.googleapis.com/wandb-production.appspot.com/buio/SQUAD/184b7gum/model-best.h5?Expires=1614708308&GoogleAcces

In [ ]:
import matplotlib.pyplot as plt

predictions = model.predict(x_eval,verbose=1)
sampled_start = np.argmax(predictions[0], axis=-1)
sampled_end = np.argmax(predictions[1], axis=-1)
plt.figure(figsize=(30,30))
plt.plot(y_eval[0], y_eval[1], ".")
plt.plot(sampled_start, sampled_end,"*")

In [58]:
!gsutil cp gs://squad_squad/evaluate.py ./evaluate.py
!chmod +x ./evaluate.py

Copying gs://squad_squad/evaluate.py...
/ [1 files][ 10.3 KiB/ 10.3 KiB]                                                
Operation completed over 1 objects/10.3 KiB.                                     


In [ ]:
# preprocess dev set
from tqdm.notebook import tqdm
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json -O test_set.json

with open("test_set.json", "r") as f:
    json_file = json.load(f)
data = json_file["data"]

rows = []
for document in data:
  for par in document['paragraphs']:
    for qas in par['qas']:

      if len(qas['answers']) == 0: #no answer
          ans_start = -1
          ans_end = -1
          ans_text = ""
      else:
          ans_start = qas['answers'][0]['answer_start']
          ans_end = ans_start + len(qas['answers'][0]['text'])
          ans_text = qas['answers'][0]['text']
          
      rows.append({
        'id' : qas['id'],
        'title': document["title"],
        'passage': par['context'],
        'question' : qas['question'],
        'answer_idx' : (ans_start, ans_end),
        'answer_text' : ans_text
      })


df_dev = pd.DataFrame(rows)

def preprocess_bert(text):
    tokenized_text = tokenizer(list(text), return_offsets_mapping=True)

    rows_out  = [{'input_ids': tokenized_text.input_ids[i],
                  'offsets': tokenized_text.offset_mapping[i]} for i in range(len(text))]

    return rows_out

def labeling(df):
    skip = []
    ans_token_start = []
    ans_token_end = []
    input_word_ids = []
    input_type_ids = []
    input_mask = []
    context_token_to_char = []

    for id in tqdm(df.index):

        answer = " ".join(str(df.loc[id]['answer_text']).split())
        tokenized_context = df.loc[id]['passage']
        tokenized_question = df.loc[id]['question']

        # mark all the character indexes in context that are also in answer     
        is_char_in_ans = [0] * len(df_dev.loc[id]['passage']) ###### BERT/DEV
        for idx in range(*df.loc[id]['answer_idx']):
            is_char_in_ans[idx] = 1


        ans_token_idx = []
        if df.loc[id]['answer_idx'] == (-1,-1):
            ans_token_idx.append(0)
        else:
        # find all the tokens that are in the answers
            for idx, (start, end) in enumerate(tokenized_context["offsets"]): #start is index of the first character of the word, end is the index of the last character of the word
                if sum(is_char_in_ans[start:end]) > 0:
                    ans_token_idx.append(idx)
            if len(ans_token_idx) == 0:
                skip.append(id)
                continue
        # create inputs as usual
        input_ids = tokenized_context['input_ids'] + tokenized_question['input_ids'][1:] #removing CLS from the beginning of the question 
        token_type_ids = [0] * len(tokenized_context['input_ids']) + [1] * len(tokenized_question['input_ids'][1:])
        attention_mask = [1] * len(input_ids)
        padding_length = max_seq_length - len(input_ids)
        
        # add padding if necessary
        if padding_length > 0:
            input_ids = input_ids + ([0] * padding_length)
            attention_mask = attention_mask + ([0] * padding_length)
            token_type_ids = token_type_ids + ([0] * padding_length)
        elif padding_length < 0:
            skip.append(id)
            continue
        input_word_ids.append(np.array(input_ids))
        input_type_ids.append(np.array(token_type_ids))
        input_mask.append(np.array(attention_mask))
        context_token_to_char.append(np.array(tokenized_context["offsets"]))
        ans_token_start.append(ans_token_idx[0])
        ans_token_end.append(ans_token_idx[-1])

    df = df.drop(skip)
    df['input_word_ids'] = input_word_ids
    df['input_type_ids'] = input_type_ids
    df['input_mask'] = input_mask
    df['context_token_to_char'] = context_token_to_char
    df['ans_token_start'] = ans_token_start
    df['ans_token_end'] = ans_token_end
 
    return df, skip

# pre-process passage and question text
df_dev = df_dev.set_index('id')
df_bert_dev = df_dev.copy()

df_bert_dev['passage'] = preprocess_bert(df_dev['passage'])
df_bert_dev['question'] = preprocess_bert(df_dev['question'])

df_bert_dev, skipped = labeling(df_bert_dev)
df_bert_dev.head(1)

In [62]:
x_test = [np.stack(df_bert_dev["input_word_ids"]),np.stack(df_bert_dev["input_mask"]),
                  np.stack(df_bert_dev["input_type_ids"])]


In [63]:
predictions = model.predict(x_test, verbose=1)

369/369 [==============================] - 233s 627ms/step


In [99]:
len(df_dev), len(skipped), len(predictions[0])

(11873, 82, 11791)

In [102]:
num_samples = len(predictions[0])

start, end = list(np.argmax(predictions, axis=-1).squeeze())
lines_c = 0
with open("dev_predictions.txt","w") as out:
    out.write("{")
    for id in skipped:
        out.write(f'''"{id}": "42",\n''')

    for ans_idx in range(num_samples):
        # no answer
        if end[ans_idx] == 0:
            if ans_idx == num_samples-1:
                out.write(f'''"{df_bert_dev.index[ans_idx]}": ""''')
            else:
                out.write(f'''"{df_bert_dev.index[ans_idx]}": "",\n''')

        # extract answer text
        else:
            predicted_ans = tokenizer.decode(df_bert_dev.iloc[ans_idx]['passage']["input_ids"][start[ans_idx] : end[ans_idx]+1]).replace("\n"," ")
            if ans_idx == num_samples-1:
                out.write(f'''"{df_bert_dev.index[ans_idx]}": "{predicted_ans.replace('"',"")}"''')
            else:
                out.write(f'''"{df_bert_dev.index[ans_idx]}": "{predicted_ans.replace('"',"")}",\n''')

    out.write("}")

In [104]:
!python3 evaluate.py test_set.json dev_predictions.txt

{
  "exact": 78.21106712709509,
  "f1": 81.65605473974477,
  "total": 11873,
  "HasAns_exact": 75.64102564102564,
  "HasAns_f1": 82.54088021676621,
  "HasAns_total": 5928
}


In [81]:
evaluation = !python3 evaluate.py test_set.json dev_predictions.txt
print(evaluation)
if ENABLE_WANDB:
    wandb.log({"Evaluate": wandb.Html("<pre>"+str(evaluation)+"<pre>", inject=False)})

['Traceback (most recent call last):', '  File "evaluate.py", line 272, in <module>', '    main()', '  File "evaluate.py", line 236, in main', '    preds = json.load(f)', '  File "/usr/lib/python3.7/json/__init__.py", line 296, in load', '    parse_constant=parse_constant, object_pairs_hook=object_pairs_hook, **kw)', '  File "/usr/lib/python3.7/json/__init__.py", line 348, in loads', '    return _default_decoder.decode(s)', '  File "/usr/lib/python3.7/json/decoder.py", line 337, in decode', '    obj, end = self.raw_decode(s, idx=_w(s, 0).end())', '  File "/usr/lib/python3.7/json/decoder.py", line 353, in raw_decode', '    obj, end = self.scan_once(s, idx)', "json.decoder.JSONDecodeError: Expecting ',' delimiter: line 2 column 34 (char 66)"]


In [ ]:
import matplotlib.pyplot as plt

#predictions = model.predict(x_test)
sampled_start = np.argmax(predictions[0], axis=-1)
sampled_end = np.argmax(predictions[1], axis=-1)

plt.figure(figsize=(30,30))
plt.plot(y_test[0], y_test[1], ".")
plt.plot(sampled_start, sampled_end,"*")

In [ ]:
# inference with custom context and question

def custom_inference(context, question):
    preprocessed_context = " ".join(str(context).split())
    preprocessed_question = " ".join(str(question).split())
    tokenized_context = tokenizer(preprocessed_context)
    tokenized_question = tokenizer(preprocessed_question)
    input_ids = tokenized_context["input_ids"] + tokenized_question["input_ids"][1:]
    token_type_ids = [0] * len(tokenized_context["input_ids"]) + [1] * len(tokenized_question["input_ids"][1:])
    attention_mask = [1] * len(input_ids)
    padding_length = max_seq_length - len(input_ids)
    if padding_length > 0:
        input_ids = input_ids + ([0] * padding_length)
        attention_mask = attention_mask + ([0] * padding_length)
        token_type_ids = token_type_ids + ([0] * padding_length)
    else:
        print("Error! The input is too long")
    input_word_ids = np.array(input_ids)
    input_mask = np.array(attention_mask)
    input_type_ids = np.array(token_type_ids)
    x = [np.expand_dims(input_word_ids, axis =0), np.expand_dims(input_mask, axis = 0), 
         np.expand_dims(input_type_ids,axis=0)]
    predictions = model.predict(x)
    start, end = list(np.argmax(predictions, axis=-1).squeeze())
    predicted_ans = tokenizer.decode(tokenized_context["input_ids"][start : end+1])
    return predicted_ans

In [ ]:
context = "Thomas Cruise Mapother IV (born July 3, 1962) is an American actor and producer. He has received various accolades for his work, including three Golden Globe Awards and three nominations for Academy Awards. He is one of the highest-paid actors in the world. His films have grossed over $4 billion in North America and over $10.1 billion worldwide, making him one of the highest-grossing box office stars of all time. Cruise began acting in the early 1980s and made his breakthrough with leading roles in the comedy film Risky Business (1983) and action drama film Top Gun (1986). Critical acclaim came with his roles in the drama films The Color of Money (1986), Rain Man (1988), and Born on the Fourth of July (1989). For his portrayal of Ron Kovic in the latter, he won a Golden Globe Award and received a nomination for the Academy Award for Best Actor. As a leading Hollywood star in the 1990s, he starred in several commercially successful films, including the drama A Few Good Men (1992), the thriller The Firm (1993), the horror film Interview with the Vampire (1994), and the romance Jerry Maguire (1996). For his role in the latter, he won a Golden Globe Award for Best Actor and received his second Academy Award nomination."
question = "What was the first film Tom Cruise acted in?"


predicted_answer = custom_inference(context, question)
print(predicted_answer)